# Chatbot de Recommandation de Films
>**Infos :**\
**Groupe :** Léonie LECAM, Quentin DEGIOVANNI, Matteo COUCHOUD\
**Année universitaire :** 2023-2024 SEMESTRE 1\
**Classe :** ING 3 IA groupe B\
**Matière :** Natural Language Processing



## I. Gestion du Dataset
On commende par importer les libraries et pré-process le dataset.

### Imports :
Pour pré-process le dataset.

### Installation des librairies :

In [ ]:
!pip install datasets
!pip install kaggle

### Gestion des imports

In [ ]:
from datasets import load_dataset
import pandas as pd
import nltk as nltk


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

### Récupération du Dataset
On utilise le [MPST Movie Dataset](https://www.kaggle.com/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags) trouvé sur Kaggle.

Nous n'utiliserons pas ce dataset à des fins de _fine-tuning_ du LLM que nous importons plus bas.\
Le dataset sera utilisé comme base de données de films connus par le chatbot, pour qu'il puisse recommander des titres de films à partir des préférences de l'utilisateur.

In [ ]:
!kaggle datasets download -d cryptexcode/mpst-movie-plot-synopses-with-tags

In [ ]:
dataset = load_dataset("csv", data_files="mpst_full_data.csv",download_mode="force_redownload")
df = pd.DataFrame(dataset['train'])

# On supprime les colonnes inutiles comme 'split', 'imdb_id'
df = df.drop(columns=['split', 'imdb_id',"synopsis_source"])



In [ ]:
# Pour chaque ligne du dataframe, on concatène le contenu des colonnes plot_synopsis et tags

df['description'] = df['plot_synopsis'] + ' ' + df['tags'] + ' ' + df['title']
# On supprime les colonnes plot_synopsis et tags
df = df.drop(columns=['plot_synopsis', 'tags'])
display(df.head())


In [ ]:
# On retire les stopwords, la ponctuation, les lettres capitales ainsi que les nombres et les mots de moins de 3 lettres
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

def clean_text_description(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    text = ' '.join([word for word in text.split() if word.isalpha()])
    text = ' '.join([word for word in text.split() if len(word) > 2])
    return text

# Pour la colonne description
df['description'] = df['description'].apply(clean_text_description)


In [ ]:
# On display les dataframes pour vérifier que tout est ok
display(df)


In [ ]:
# On retire les lignes qui ont un titre ou une description vide
df = df.dropna()


In [ ]:
# On display le dataframe pour vérifier que tout est ok
display(df)

## II. Algorithme de recommandation de films
Dans cette partie, nous définissions une classe _MovieRecommender_ qui permet :
- De vectoriser chaque description de titre
- De calculer les distances entre chaque vecteurs et les stocker dans une matrice
- Prendre en compte la phrase de l'utilisateur afin de calculer sa distance avec les descriptions de films
- Retourner jusqu'à 3 films proches de la description donnée par l'utilisateur.

### Définition de la classe _MovieRecommender_

In [ ]:
# On créé une fonction qui prend en entrée un array, parcours chaque élément et le retire s'il est déjà présent dans l'array
def remove_duplicates(array):
    new_array = []
    for element in array:
        if element not in new_array:
            new_array.append(element)
    return new_array

In [ ]:
# We create a class for the MovieRecommender :
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

class MovieRecommender:

    def __init__(self, df):
        self.df = df
        self.tdfidf_vectorizer = TfidfVectorizer(stop_words='english')
        print('test')
        # On précalcule la matrice de similarité cosinus
        self.tfidf_matrix = self.tdfidf_vectorizer.fit_transform(df['description'])
        self.cosine_sim = linear_kernel(self.tfidf_matrix, self.tfidf_matrix)

    def get_recommendations(self, user_input):
        user_tfidf = self.tdfidf_vectorizer.transform([user_input])
        user_cosine_sim = linear_kernel(user_tfidf, self.tfidf_matrix)
        # Les indices des 3 films les plus similaires à la description de l'utilisateur
        similar_indices = user_cosine_sim.argsort()[0][-4:-1]
        # On récupère les scores de similarité
        similar_scores = user_cosine_sim[0][similar_indices]
        # On récupère les titres des films dans un array
        similar_titles = self.df['title'].iloc[similar_indices].values
        # On retire les éventuels doublons
        similar_titles = remove_duplicates(similar_titles)
        
        return similar_titles, similar_scores

### Initialisation d'un _MovieRecommender_
A la création d'une instance de _MovieRecommender_, le classe calcule la matrice de similarité entre toutes les descriptions (tags+synopsis+titre) des films.

In [ ]:
# On crée une instance de la classe MovieRecommender
movie_recommender = MovieRecommender(df)

### DEBUG : Test d'une recommandation :

In [ ]:
# On teste la méthode get_recommendations

recommendations,scores = movie_recommender.get_recommendations("a film with pirates in the caribbean")
print(recommendations)

## III. Gestion du LLM
Comme LLM, nous utiliserons FLAN-T5 dans sa taille "_large_" de Google.\
Le LLM sera utilisé dans la partie conversationnelle du chatbot.\

Pour éviter toute sortie tronquée du LLM, la taille maximale de la sortie est définie à 200 tokens.

In [ ]:
# We first need to import Flan-t5-base model and tokenizer
model_id = "google/flan-t5-large"

# We load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# We load the model
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# On veut spécifier au modèle la longueur maximale de la séquence de sortie
# On va donc créer un dictionnaire de paramètres
# On veut que la longueur maximale de la séquence de sortie soit de 100 tokens

params = {
    'max_length': 200,
    'temperature': 0,
    'repetition_penalty': 2.5,
}



### Définition d'une méthode _Query()_
Cette méthode permet d'automatiser la tokenization d'un prompt, la génération et le décodage de sa réponse par le LLM.

In [ ]:
# On défini une fonction query qui prend en paramètre une requête et qui renvoie une réponse
def query(question):
    input = question
    input_ids = tokenizer.encode(input, return_tensors="pt")
    res = model.generate(input_ids, **params)
    answer = tokenizer.decode(res[0], skip_special_tokens=True)
    return answer

### Traitement des prompts de l'utilisateur
Dans cette partie, on défini un ensemble de fonctions.
Ces fonctions ont pour but de prendre le prompt de l'utlisateur et d'utiliser les fonctionnalité de conversation et compréhension du LLM afin de déterminer si :
- L'utilisateur parle de film/demande une recommandation
- Lui renvoyer un message d'erreur lorsque le LLM ne comprends pas sa demande/aucun film n'est trouvé.

#### Fonction de traitement : l'utilisateur parle-t-il de films ?

In [ ]:


def is_about_movies(text):
    answer = query("""Text: \”"""+text+"""\\nGiven the text, tell if the text is about movies. Tell 'NO' if the user did not talk about movies, and 'YES' if the user talked about movies.""")
    print(answer)
    if(answer == "Yes" or answer == "yes" or answer == "YES"):
        return True
    else:
        return False

    

In [ ]:
intro_sentence = query("Hello, I am a movie recommendation chatbot. Please describe what kind of film you would like to watch ! Repeat what is written above this last sentence.")
print(intro_sentence)

#### Fonction Array->String
Cette fonction permet de passer de l'array retourné par la méthode _get_recommendations()_ de _MovieRecommender_ à une chaine de charactères mise en forme.

In [ ]:
# On créé une fonction pour passer d'un array à une string
def array_to_string(array):
    string = ""
    for i in range(len(array)):
        string += array[i] + ", "
    return str(string)

#### Fonctions encapsulant le traitement du prompt de l'utilisateur
__process_text()__: Cette fonction permet de demander, à chaque prompt de l'utilisateur, de traiter la demande afin d'en comprendre l'intention (demande de recommandation de films) et d'en extraire les données permettant au chatbot de remplir sa fonction première.


In [ ]:
# On va maintenant créer une fonction qui va traiter le texte donné au LLM, et lui demander de générer un texte en sortie
def process_text(text):
    # on cherche à savoir si l'utilisateur parle de films
    if(is_about_movies(text)):

        movies,scores = movie_recommender.get_recommendations(text)
        print(array_to_string(movies))
        movie_list = array_to_string(movies)
        res = query("""
                    QUERY : Generate a text that lists the following movie titles: The Hobbit, The Lord of the Rings, The Matrix.
                    ANSWER : Here are some movie recommendations based on your preferences : The Hobbit, The Lord of the Rings, The Matrix.

                    QUERY : Generate a text that lists the following movie titles: Harry Potter and the Prisonner of Askaban, Dune.
                    ANSWER : Here are some movie recommendations based on your preferences : Harry Potter and the Prisonner of Askaban, Dune.

                    QUERY : QUERY : Generate a text that lists the following movie titles:"""+movie_list+""".
                    ANSWER : 
                    """)    
    else:
        # On dit à l'utilisateur qu'on n'a pas compris
        res = "Sorry, I could not understand what you said. I am a movie recommendation chatbot. Please describe what kind of film you would like to watch !"
    return res


# IV. Interface Utilisateur


In [ ]:
!pip install tk


In [ ]:
# On importe tkinter
import tkinter as tk


In [ ]:

class ChatbotGUI:
  def __init__(self):
    
    # On crée une fenêtre
    self.root = tk.Tk()
    self.root.geometry("")

    # On donne une couleur de fond à la fenêtre
    self.root.configure(bg="#393c4d")
    self.root.title("Chatbot - Projet par Léonie LECAM, Quentin DEGIOVANNI et Matteo COUCHOUD - CY-TECH 2023-2024")

    # Create the chatbot's text area
    self.text_area = tk.Text(self.root, bg="#21232e", fg="white", width=100, height=21, font=('Montserrat',13), wrap=tk.WORD, padx=10, pady=10)
    self.text_area.tag_configure("bold", font=("Montserrat",13,'bold'))
    #text_area.pack()
    self.text_area.grid(row=0,column=0,columnspan=2)

    # Create the user's input field
    self.input_field = tk.Entry(self.root, width=89, font=('Montserrat',13), relief=tk.FLAT, border=0, bg="#21232e", fg="white")
    #input_field.pack()
    self.input_field.grid(row=1,column=0,padx=10,pady=10)

    # Create the send button
    self.send_button = tk.Button(self.root, text="Send", command=lambda: self.send_message(), font=('Montserrat',13,'bold'), border=0)
    #send_button.pack()
    self.send_button.grid(row=1,column=1)

    # On ajoute une introduction au chatbot dans la fenêtre 
    self.text_area.insert(tk.END, f"Chatbot: ", "bold")
    self.text_area.insert(tk.END, f"{intro_sentence}\n")

    # On lance la fenêtre
    self.root.mainloop()

  def send_message(self):
    # Get the user's input
    user_input = self.input_field.get()

    # Clear the input field
    self.input_field.delete(0, tk.END)

    # Generate a response from the chatbot
    response = process_text(user_input)

    # Display the response in the chatbot's text area
    self.text_area.insert(tk.END, f"\nPrompt: ", "bold")
    self.text_area.insert(tk.END, f"{user_input}\n\n")
    self.text_area.insert(tk.END, f"Chatbot: ", "bold")
    self.text_area.insert(tk.END, f"{response}\n")

In [31]:
# On lance l'interface du chatbot
chatbot_instance = ChatbotGUI()

NO
YES
Star Wars: Episode II - Attack of the Clones, Star Wars: Episode I - The Phantom Menace, Star Wars: Episode III - Revenge of the Sith, 
